In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from google.cloud import bigquery
import matplotlib.pyplot as plt
import os

In [ ]:
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')

# Preprocessing

### fill the missing 'Province' with 'Country'

In [ ]:
train.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)
test.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)

In [ ]:
for i in range(len(train)):
    if train.Province[i] is np.NaN:
        train.Province[i] = train.Country[i]
for i in range(len(test)):
    if test.Province[i] is np.NaN:
        test.Province[i] = test.Country[i]

### transform date to days

In [ ]:
def get_days(x):
    x = datetime.strptime(x,'%Y-%m-%d')
    first = datetime.strptime('2020-01-01','%Y-%m-%d')
    diff = (x - first).days
    return diff

In [ ]:
train['Days_After_1stJan'] = train.Date.apply(lambda x :get_days(x))
test['Days_After_1stJan'] = test.Date.apply(lambda x :get_days(x))

In [ ]:
train.head()

In [ ]:
test.head()

### Add Date Features

In [ ]:
def get_dt(x):
    x = datetime.strptime(x,'%Y-%m-%d')
    return x
def get_dayofweek(x):
    return x.dayofweek
def get_month(x):
    return x.month
def get_day(x):
    return x.day

In [ ]:
train['Date'] = train.Date.apply(lambda x:get_dt(x))
test['Date'] = test.Date.apply(lambda x:get_dt(x))
train['Dayofweek'] = train.Date.apply(lambda x:get_dayofweek(x))
test['Dayofweek'] = test.Date.apply(lambda x:get_dayofweek(x))
train['Month'] = train.Date.apply(lambda x:get_month(x))
test['Month'] = test.Date.apply(lambda x:get_month(x))
train['Day'] = train.Date.apply(lambda x:get_day(x))
test['Day'] = test.Date.apply(lambda x:get_day(x))

# Merge population information for each country

In [ ]:
country_info = pd.read_csv('/kaggle/input/population/population_by_country_2020.csv')

### Select wanted features from table

In [ ]:
population = pd.DataFrame(country_info.iloc[:,[0,1,4,5,6,8,9]])
population.columns = ['Country','Population','Density','Land_Area','Migrants','MedAge','UrbanPopRate']

### fill NAN with median and mode

In [ ]:
for i in range(len(population)):
    if np.isnan(population.Migrants[i]):
        population.Migrants[i] = np.nanmedian(population.Migrants)
    if population.MedAge[i] == 'N.A.':
        population.MedAge[i] = 19
    if population.UrbanPopRate[i] == 'N.A.':
        population.UrbanPopRate[i] = '57%'

### Collect data for countries without population info manually

In [ ]:
kosovo = pd.DataFrame([['Kosovo'],[2000700],[168],[10887],[0],[19],['57%']])
kosovo = kosovo.T
kosovo.columns = population.columns


population = population.append(kosovo)

In [ ]:
westbank = pd.DataFrame([['West Bank and Gaza'],[2697687],[485],[5559],[0],[19],['57%']])
westbank = westbank.T
westbank.columns = population.columns

population = population.append(westbank)

In [ ]:
DP = pd.DataFrame(['Diamond Princess',2666,191522,0.01392,2666,19,'100%'])
DP = DP.T
DP.columns = population.columns

population = population.append(DP)

In [ ]:
Shangdan = pd.DataFrame(['MS Zaandam',1432,189618,0.007552,1432,19,'100%'])
Shangdan = Shangdan.T
Shangdan.columns = population.columns

population = population.append(Shangdan)

### Change names of countries to match names in train dataframe

In [ ]:
Congo = population[population.Country == 'Congo']
Congo['Country'] = 'Congo (Brazzaville)'
new1 = Congo.copy()
Congo['Country'] = 'Congo (Kinshasa)'
new2 = Congo.copy()

population = population.append(new1)
population = population.append(new2)

In [ ]:
population.Country[population.Country == 'United States'] = 'US'
population.Country[population.Country == 'Taiwan'] = 'Taiwan*'
population.Country[population.Country == 'South Korea'] = 'Korea, South'
population.Country[population.Country == 'Côte d\'Ivoire'] = 'Cote d\'Ivoire'
population.Country[population.Country == 'Czech Republic (Czechia)'] = 'Czechia'
population.Country[population.Country == 'Myanmar'] = 'Burma'
population.Country[population.Country == 'St. Vincent & Grenadines'] = 'Saint Vincent and the Grenadines'
population.Country[population.Country == 'Saint Kitts & Nevis']  = 'Saint Kitts and Nevis'
population.Country[population.Country == 'Sao Tome & Principe']  = 'Sao Tome and Principe'

### merge df

In [ ]:
train = pd.merge(train,population,left_on = 'Country',right_on='Country',how='left')
test = pd.merge(test,population,left_on = 'Country',right_on='Country',how = 'left')

In [ ]:
train.head()

In [ ]:
test.head()

# Add weather information

In [ ]:
temperature = pd.read_csv('/kaggle/input/weather-data-for-covid19-data-analysis/training_data_with_weather_info_week_4.csv')
temperature.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)

In [ ]:
# fill NAN of Province with Country name
for i in range(len(temperature)):
    if temperature.Province[i] is np.NaN:
        temperature.Province[i] = temperature.Country[i]

In [ ]:
#delete useless features 
del temperature['ConfirmedCases']
del temperature['Fatalities']
del temperature['country+province']
del temperature['day_from_jan_first']

In [ ]:
#transform date to datetime type
temperature.Date = temperature.Date.apply(lambda x:get_dt(x))

### since the weather data only contains information up to 2020-04-08,I delete the data of last 4days in the train dataframe

In [ ]:
day1 = datetime.strptime('2020-04-09','%Y-%m-%d')
day2 = datetime.strptime('2020-04-10','%Y-%m-%d')
day3 = datetime.strptime('2020-04-11','%Y-%m-%d')
day4 = datetime.strptime('2020-04-12','%Y-%m-%d')

In [ ]:
index_delete = []
for i in range(len(train)):
    if (train.Date[i] == day1) or (train.Date[i] == day2) or (train.Date[i] == day3) or (train.Date[i] == day4):
        index_delete.append(i)
        
train = (train.drop(index = index_delete)).reset_index(drop = True)

In [ ]:
train = pd.merge(train,temperature,on=['Country','Province','Date'],how='left')

In [ ]:
#fill NAN
train['Lat'][train['Lat'].isnull()] = np.nanmedian(train['Lat'])
train['Long'][train['Long'].isnull()] = np.nanmedian(train['Long'])
train['temp'][train['temp'].isnull()] = np.nanmedian(train['temp'])
train['min'][train['min'].isnull()] = np.nanmedian(train['min'])
train['max'][train['max'].isnull()] = np.nanmedian(train['max'])
train['slp'][train['slp'].isnull()] = np.nanmedian(train['slp'])
train['dewp'][train['dewp'].isnull()] = np.nanmedian(train['dewp'])
train['rh'][train['rh'].isnull()] = np.nanmedian(train['rh'])
train['ah'][train['ah'].isnull()] = np.nanmedian(train['ah'])
train['stp'][train['stp'].isnull()] = np.nanmedian(train['stp'])
train['wdsp'][train['wdsp'].isnull()] = np.nanmedian(train['wdsp'])
train['prcp'][train['prcp'].isnull()] = np.nanmedian(train['prcp'])
train['fog'][train['fog'].isnull()] = np.nanmedian(train['fog'])

### Use auto arima to forecast weather data

In [ ]:
!pip install pmdarima

In [ ]:
import pmdarima

In [ ]:
#the outlier 'inf' will make the auto_arima come to an error,so it's replaced by 0
train.replace(np.inf,0,inplace=True)

In [ ]:
#Using arima to predict the weather information for future
date_pred_df = pd.DataFrame(sorted(list(set(test.Date))),columns=['Date'])
date_pred_df = date_pred_df[7:]
date_pred_df.reset_index(inplace = True)
del date_pred_df['index']

nperiods = (datetime.strptime('2020-05-14','%Y-%m-%d')-datetime.strptime('2020-04-08','%Y-%m-%d')).days

weather_feature = ['temp', 'min', 'max', 'stp', 'slp', 'dewp', 'rh', 'ah','wdsp', 'prcp', 'fog']
weather_pred = pd.DataFrame(columns=['Date','temp', 'min', 'max', 'stp', 'slp', 'dewp', 'rh', 'ah','wdsp', 'prcp', 'fog'])
for prov in list(set(train.Province)):
    df = train[train.Province == prov]
    province_pred = date_pred_df.copy()
    for feature in weather_feature:
        ts = df[feature]
        model = pmdarima.auto_arima(ts)
        pred = model.predict(n_periods = nperiods)
        province_pred[feature] = pred
    province_pred['Province'] = prov
    weather_pred = pd.concat([weather_pred,province_pred],axis = 0)

In [ ]:
test

In [ ]:
for i in range(len(test)):
    if test.Date[i]<datetime.strptime('2020-04-09','%Y-%m-%d'):
        test.drop(i,inplace=True)

In [ ]:
#get longitude and latitude dataframe and merge it into test df
df_longlat = pd.DataFrame(columns = ['Province','Lat','Long'])
for i in range(len(train)):
    if train.Province[i] not in list(df_longlat.Province):
        df_longlat = df_longlat.append(train.iloc[i][['Province','Lat','Long']])
        
test = pd.merge(test,df_longlat,on = 'Province',how = 'left')

In [ ]:
#adding weather feature to test data
test = pd.merge(test,weather_pred,on = ['Province','Date'],how = 'left')

In [ ]:
train.head()

In [ ]:
test.head()

# Adding Index of Beds per person

In [ ]:
API_beds = pd.read_csv('/kaggle/input/newest-bed-api-for-each-country/Newest_avg_bed_API.csv')

In [ ]:
#merge
train = pd.merge(train,API_beds,left_on='Country',right_on='Country',how='left')
test = pd.merge(test,API_beds,left_on='Country',right_on='Country',how='left')

In [ ]:
#fill NAN
train.API_beds[train.API_beds.isnull()] = np.nanmedian(train.API_beds)
test.API_beds[test.API_beds.isnull()] = np.nanmedian(test.API_beds)

In [ ]:
train.head()

In [ ]:
test.head()

# Set train data for XGBoost

In [ ]:
X = train.copy()
X_test = test.copy()

Province_set = set(X.Province)
Country_set = set(X.Country)

X = pd.concat([X,pd.get_dummies(X.Country)],axis=1)
X_test = pd.concat([X_test,pd.get_dummies(X_test.Country)],axis=1)
X = pd.concat([X,pd.get_dummies(X.Province)[Province_set - Country_set]],axis=1)
X_test = pd.concat([X_test,pd.get_dummies(X_test.Province)[Province_set - Country_set]],axis=1)

y_confirm = X.ConfirmedCases
y_fata = X.Fatalities


In [ ]:
del X['ConfirmedCases']
del X['Fatalities']
del X['Id_x']
del X['Date']
del X['Id_y']
del X['Province']
del X['Country']

In [ ]:
ForecastId = X_test.ForecastId

In [ ]:
def get_percent(x):
    x = str(x)
    x = x.strip('%')
    x = float(x)/100
    return x

In [ ]:
X['UrbanPopRate'] = X.UrbanPopRate.apply(lambda x:get_percent(x))
X_test['UrbanPopRate'] = X_test.UrbanPopRate.apply(lambda x:get_percent(x))

In [ ]:
del X_test['ForecastId']
del X_test['Date']
del X_test['Province']
del X_test['Country']

In [ ]:
X = pd.DataFrame(X,dtype=float)
X_test = pd.DataFrame(X_test,dtype=float)

In [ ]:
order = X.columns
X_test = X_test[order]

In [ ]:
X.head()

In [ ]:
X_test.head()

# XGBoost Training

In [ ]:
import xgboost as xgb

In [ ]:
reg_confirm = xgb.XGBRegressor()
reg_confirm.fit(X,y_confirm)

In [ ]:
pred_confirm = reg_confirm.predict(X_test)

In [ ]:
reg_fata = xgb.XGBRegressor()
reg_fata.fit(X,y_fata)

In [ ]:
pred_fata = reg_fata.predict(X_test)

In [ ]:
submit = pd.DataFrame(ForecastId)
submit['ConfirmedCases']=pred_confirm
submit['Fatalities']=pred_fata

In [ ]:
temp_submit_df = submit.copy()
temp_submit_df['Province'] = test.Province
temp_submit_df['Date'] = test['Date']
del temp_submit_df['ForecastId']

In [ ]:
original_train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
new_test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')

In [ ]:
original_train.Date = original_train.Date.apply(lambda x:get_dt(x))
original_train.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)
for i in range(len(original_train)):
    if original_train.Province[i] is np.NaN:
        original_train.Province[i] = original_train.Country[i]
        
for i in range(len(original_train)):
    if original_train.Date[i]<datetime.strptime('2020-04-02','%Y-%m-%d') or original_train.Date[i]>datetime.strptime('2020-04-08','%Y-%m-%d'):
        original_train.drop(i,inplace=True)
        
del original_train['Id']
del original_train['Country']

In [ ]:
final = pd.concat([temp_submit_df,original_train],axis = 0)
final = final.sort_values(by=['Province','Date'])
final_submit = pd.DataFrame(new_test.ForecastId,columns=['ForecastId'])
final_submit['Confirmed'] = final.ConfirmedCases.values
final_submit['Fatalities'] = final.Fatalities.values

In [ ]:
final_submit

In [ ]:
final_submit.to_csv('/kaggle/working/submission.csv',index=False)